In [7]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# %matplotlib inline
# import matplotlib.pyplot as plt

train_data_dir = "train"
validation_data_dir = "val"
batch_size = 16
epochs = 5000

img_width, img_height = 448, 448

### Build the network 
img_input = Input(shape=(img_width, img_height, 3))
x = Conv2D(24, (5,5), strides=(2,2), activation='relu')(img_input)
x = Conv2D(32, (5,5), strides=(2,2), activation='relu')(x)
x = Conv2D(64, (5,5), strides=(2,2), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(128, (3,3), strides=(1,1), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = img_input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 448, 448, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 222, 222, 24)      1824      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 109, 109, 32)      19232     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 53, 53, 64)        51264     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
__________

In [8]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
vertical_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
shuffle=True,
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
batch_size = batch_size,
target_size = (img_height, img_width),
shuffle=True,
class_mode = "categorical")

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("model.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=500, verbose=1, mode='auto')


Found 230066 images belonging to 2 classes.
Found 9227 images belonging to 2 classes.


In [9]:

model_final.fit_generator(
    train_generator,
#     steps_per_epoch=nb_train_samples // batch_size,
    steps_per_epoch=200,
    epochs=epochs,
    validation_data=validation_generator,
#     validation_steps=nb_validation_samples // batch_size,
    validation_steps=50,
    callbacks = [checkpoint, early])

Epoch 1/5000
180/200 [==========================>...] - ETA: 13s - loss: 0.6408 - acc: 0.7257

KeyboardInterrupt: 